In [0]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from PIL import Image
import io
from tempfile import mkdtemp
import pickle as pkl
import tarfile
import scipy
import time

import operator as op
from functools import reduce

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

In [0]:
# read label file
tar_file = tarfile.open("/content/drive/My Drive/data_filelist_places365-standard.tar")
f = tar_file.extractfile("places365_val.txt")
content = f.read().decode(encoding="utf-8")

In [0]:
val_label = tf.convert_to_tensor(np.array([int(i.split(' ')[1]) for i in content.split('\n')]), dtype = tf.float64)

In [0]:
# input data_x and data_y should be numpy array
class multi_svm_model:
    def __init__(self, n_class, dimension, learning_rate=1e-2, regularization=1):
        self.learning_rate=learning_rate
        self.n_class=n_class
        self.dimension=dimension
        self.w=[]
        self.b=[]
        self.model_y=[]
        self.loss=[]
        self.hinge_loss=[]
        self.regularization_loss=[]
        #prediction=[]
        self.gt_y=[]
        self.lookup_class=dict()
        self.w_model=np.random.normal(loc=0,scale=1,size=(self.ncr(n_class,2), dimension))
        self.b_model=np.random.normal(loc=0,scale=1,size=(self.ncr(n_class,2),1))
        self.lookup_matrix=np.zeros((n_class, self.ncr(n_class,2)),dtype=np.float32)
        self.batch_x=tf.placeholder(tf.float32,shape=(None,dimension),name="batch_x")
        self.batch_y=tf.placeholder(tf.float32,shape=(None,1),name="batch_y")
        self.w = tf.Variable(tf.random_uniform([self.ncr(n_class,2), self.dimension]))
        self.b = tf.Variable(tf.random_uniform([self.ncr(n_class,2),1]))
        k=0
        for i in range(n_class-1):
            for j in range(i+1,n_class):
                self.lookup_class[k]=[i,j]
                k += 1

        for i in range(n_class):
            for j in range(self.ncr(n_class,2)):
                if i in self.lookup_class[j]:
                    if self.lookup_class[j][0]==i:
                        self.lookup_matrix[i,j]=1.0
                    else:
                        self.lookup_matrix[i,j]=-1.0

        for i in range(self.ncr(n_class,2)):
            # idx is the index of all the samples svm i concerns
            
            # idx=tf.where[condition is true] 
            # tf.gather_nd(a,idx) 
            # is equivalent to a[condition is true]
            idx=tf.where(tf.keras.backend.any(tf.equal(self.batch_y,self.lookup_class[i]),1)) 
            
            # 1 x N matrix
            self.model_y.append(
                tf.tanh(
                    tf.matmul(tf.reshape(self.w[i,:],(1,dimension)),
                              tf.gather_nd(self.batch_x, idx), transpose_b=True)
                    + self.b[i,:])) 
            
            # 1 x N matrix
            self.gt_y.append(tf.reshape(self.zonp(tf.cast(
                tf.equal(tf.gather_nd(self.batch_y, idx),self.lookup_class[i][0]),
                tf.float32)),(1,-1)))
            
            self.hinge_loss.append(tf.reduce_mean(tf.maximum(0.0,1-tf.multiply(self.model_y[i],self.gt_y[i]))))
            self.regularization_loss.append(regularization * tf.norm(self.w[i,:], 2))
            self.loss.append(self.hinge_loss[i] + self.regularization_loss[i])
        self.total_loss = sum(self.loss)
        self.opt = tf.train.RMSPropOptimizer(learning_rate).minimize(self.total_loss)
        
    def ncr(self,n, r):
        r = min(r, n-r)
        numer = reduce(op.mul, range(n, n-r, -1), 1)
        denom = reduce(op.mul, range(1, r+1), 1)
        return numer // denom
    
    def zonp(self,zero_one):
        return 2*zero_one-1

    def fit(self,data_x,data_y,iter_time=100, batch_size=1000):
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            sess.run([tf.assign(self.w,self.w_model),tf.assign(self.b,self.b_model)])
            
            total = data_y.shape[0]
            lower = 0
            upper = lower + batch_size
            
            counter = 0
            while counter < iter_time:
                lower = max(0,        (lower   +batch_size) % total)
                upper = min(total,      (lower +batch_size) % total)
                if lower<upper:
                    _,loss_val = sess.run([self.opt,self.total_loss], feed_dict={self.batch_x:data_x[lower:upper],self.batch_y:data_y[lower:upper]})
                    print("iteration:"+str(counter)+" loss:"+str(loss_val))
                    counter += 1
                
            w_model,b_model = sess.run([self.w,self.b])
        self.w_model = w_model
        self.b_model = b_model
        
        return w_model,b_model
    
    def predict(self,data_x):
        result = np.argmax(np.matmul(self.lookup_matrix, np.tanh(np.matmul(self.w_model,data_x.T)) ),axis=0)
        return result.reshape(-1,1)

In [0]:
# X and y should be tensor
class SVM:
    def __init__(self, X, y, reg):
        """ Initialize the SVM attributes and initialize the weights vector to the zero vector. 
            Attributes: 
                X (tensor) : training data intputs
                y (tensor) : 1D numpy array of training data outputs
                reg (float) : regularizer parameter
                theta : 1D tensor of weights
        """
        self.X = X
        self.y = y
        self.reg = reg
        self.theta = tf.zeros([self.X.shape[1], ], dtype = tf.float64)
    
    def objective(self, X, y):
        """ Calculate the objective value of the SVM. When given the training data (self.X, self.y), this is the 
            actual objective being optimized. 
            Args:
                X (array_like) : array of examples, where each row is an example
                y (array_like) : array of outputs for the training examples
            Output:
                (float) : objective value of the SVM when calculated on X,y
        """
        theta_xy = tf.math.multiply(self.y, tf.tensordot(self.X, self.theta, axes = 1))
        hinge = tf.reduce_sum(tf.math.maximum(1-1-theta_xy, 0))
        reg = (self.reg / 2) * scipy.linalg.norm(self.theta) ** 2
        objective = hinge + reg
        
        return objective
    
    def gradient(self):
        """ Calculate the gradient of the objective value on the training examples. 
            Output:
             (vector) : 1D numpy array containing the gradient
        """
        u = tf.tensordot(self.X, self.theta, axes = 1)
        v = tf.math.multiply(self.y, u)
        v = tf.cast(tf.where(v<=1, 1., 0.), 'float64')
        y_v = (-self.y * v)[:,None]
        grad = tf.reshape((tf.transpose(y_v) @ self.X + self.theta * self.reg),[-1])

        return grad
     
    def train(self, niters=100, learning_rate=1, verbose=False):
        """ Train the support vector machine with the given parameters. 
            Args: 
                niters (int) : the number of iterations of gradient descent to run
                learning_rate (float) : the learning rate (or step size) to use when training
                verbose (bool) : an optional parameter that you can use to print useful information (like objective value)
        """
        for epoch in range(niters):
            grad = tf.transpose(self.gradient())
            self.theta = self.theta - (grad * learning_rate)
        return

    def predict(self, X):
        """ Predict the class of each label in X. 
            Args: 
                X (array_like) : array of examples, where each row is an example
            Output:
                (vector) : 1D numpy array containing predicted labels
        """
        predict = tf.tensordot(self.X, self.theta, axes = 1)
        # predict = tf.where(predict>0,1.,-1.)
        
        return predict